## HW 5
Sevgi Ozturk

### Part I

Consider the following model

\begin{align}
A \sim \mathcal{N}(A; 0, 1.2) \\
R \sim \mathcal{IG}(R; 0.4, 250) \\
x_k | x_{k-1} \sim \mathcal{N}(x_k;Ax_{k-1},R)
\end{align}

#### Draw the directed graphical model and the factor graph.
<img src = "./DGM.png" height="800" width="800"/>
<img src = "./FG.png" height="600" width="600"/>

#### Write the expression for the full joint distribution and assign terms to the individual factors on the factor graph.

\begin{eqnarray}
\phi = p(A,R,x_1=\hat x_1,x_0=\hat x_0) \propto p(x_1|x_o,A,R)p(A)p(R) = p(x_1,A,R|x_0) \\
\phi = \mathcal{N}(x_k;Ax_{k-1},R)\mathcal{N}(A; 0, 1.2)\mathcal{IG}(R; 0.4, 250) \\
\phi = exp(-\frac{1}2 \frac{x_1^2}R + x_ox_1\frac{A}R-\frac{1}2\frac{x_0^2A^2}R-\frac{1}2log2 \pi R)exp(-\frac{A^2}2.4-\frac{1}2log2.4 \pi )exp(-1.4logR-\frac{250}R-log\Gamma (0.4)+0.4log250) \\
\phi_A = exp(-\frac{A^2}{2P}) = exp(-\frac{A^2}{2.4})\\
\phi_R = exp(-\frac{1}2 \frac{x_1^2}R-\frac{logR}2-(\mathcal{v}+1)logR-\frac{\mathcal{v}}{\beta R}) = exp(- (\mathcal{v}+1+\frac{1}2)logR-\left( \frac{v}\beta+\frac{x_1^2}2 \right)\frac{1}R)\\
\phi_{X_oX_1AR} = exp(x_ox_1\frac{A}R-\frac{1}2 \frac{x_o^2A^2}R)
\end{eqnarray}

#### Derive the joint distribution p(A,R,x_o=1,x_1=-6) and create contour plot.
\begin{eqnarray}
\phi = exp(-\frac{1}2 \frac{x_1^2}R + x_ox_1\frac{A}R-\frac{1}2\frac{x_0^2A^2}R-\frac{1}2log2 \pi R)exp(-\frac{A^2}2.4-\frac{1}2log2.4 \pi )exp(-1.4logR-\frac{250}R-log\Gamma (0.4)+0.4log250) \\
log\phi = -\frac{1}2 \frac{x_1^2}R + x_ox_1\frac{A}R-\frac{1}2\frac{x_0^2A^2}R-\frac{1}2log2 \pi R -\frac{A^2}2.4-\frac{1}2log2.4 \pi -1.4logR-\frac{250}R-log\Gamma (0.4)+0.4log250) \\
log\phi = -\frac{1}2 \frac{(-6)^2}R + 1(-6)\frac{A}R-\frac{1}2\frac{1^2A^2}R-\frac{1}2log2 \pi R -\frac{A^2}2.4-\frac{1}2log2.4 \pi -1.4logR-\frac{250}R-log\Gamma (0.4)+0.4log250) \\
\end{eqnarray}

### Part II
\begin{align}
A \sim \mathcal{N}(A; 0, P) \\
R \sim \mathcal{IG}(R; \mathcal{v}, \frac{\mathcal{v}}\beta) \\
x_k | x_{k-1} \sim \mathcal{N}(x_k;Ax_{k-1},R)
\end{align}

#### Derive and implement an EM algorithm to find the MAP estimate
\begin{align}
R^* = argmax  p(R|x_o,x_1,\theta)
\end{align}

\begin{eqnarray}
KL(q(R|x_0,x_1) || p(R|x_0,x_1,\theta) = \int q(R|x_0,x_1) log \frac{q(R|x_0,x_2)}p(R|x_0,x_1) \\
KL(q(R|x_0,x_1) || p(R|x_0,x_1,\theta) = \bigg \langle log(q(R|x_0,x_1))\bigg \rangle_{q(R|x_0,x_1)} - \bigg \langle log(p(R|x_0,x_1))\bigg \rangle_{q(R|x_0,x_1)}\\
\end{eqnarray}
Second term is the energy term and we're trying to maximize this.
EM has two steps\\
E step : fix parameters, find distributions 
\begin{eqnarray}
q(R) \propto exp(\big \langle log(\phi_{x_0x_1AR})_{q(A)} \big \rangle)
= exp(-\frac{A^2}{2.4} + x_ox_1\frac{A}R-\frac{1}2 \frac{x_o^2A^2}R)
\end{eqnarray}
M step : fix q(R), find parameters 

Same is true for :
\begin{eqnarray}
q(A) \propto exp(\big \langle log(\phi_{x_0x_1AR})_{q(R)} \big \rangle)=exp(- (\mathcal{v}+1+\frac{1}2)logR-\left( \frac{v}\beta+\frac{x_1^2}2 \right)\frac{1}R + x_ox_1\frac{A}R-\frac{1}2 \frac{x_o^2A^2}R)
\end{eqnarray}